In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import pydicom
import matplotlib.pyplot as plt
import os
import shutil
import glob
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision import transforms as T
from IPython import display
import SimpleITK as sitk
from ipywidgets import interact, fixed
import random 
from tqdm import tqdm
from PIL import Image
from multiprocessing import Pool
import os

In [2]:
# 统计训练集，验证集，颈胸腰椎
def get_loc_forpatients(dir):
    namelist = []
    for path in os.listdir(dir):
        path = os.path.basename(path)
        name = path.split('_')[0]+'_'+path.split('_')[1]
        namelist.append(name)
    namelist =set(namelist)
    # 
    namelist = pd.DataFrame({'location':[name.split('_')[0] for name in namelist], 'id':[name.split('_')[1] for name in namelist]})
    namelist = namelist.drop_duplicates('id')
    return namelist

def get_disease_forpatients(dir):
    namelist = []
    for path in os.listdir(dir):
        path = os.path.basename(path)
        # name = path.split('_')[0]+'_'+path.split('_')[1]
        namelist.append(path)
    namelist =set(namelist)
    # 
    diseases = []
    for name in namelist:
        namesplit = name.split('_')
        if len(namesplit) == 6:
            disease = namesplit[2] + '_' +namesplit[3] 
        elif len(namesplit) == 7:
            disease = namesplit[2] + '_' +namesplit[3]+ '_' +namesplit[4] 
        elif len(namesplit) == 8:
            disease = namesplit[2] + '_' +namesplit[3]+ '_' +namesplit[4]+ '_' +namesplit[5]             
        elif len(namesplit) == 5:
            disease = namesplit[2]
        else:
            print()
            print(name)
        diseases.append(disease)
    namelist = pd.DataFrame({'disease':diseases, 'id':[name.split('_')[1] for name in namelist]})
    namelist = namelist.drop_duplicates('id')
    return namelist

In [41]:
train_namelist = get_loc_forpatients("./DiffusionSpinalMRISynthesis/Data_MRI/train_spinal_MRI")
train_namelist

,location,id
0,L,3015374
1,L,2239647
2,S,3711733
3,C,2982274
4,L,3767305
...,...,...
806,L,3362561
807,L,3408340
808,L,1600837
809,T,3059169


In [43]:
train_namelist.value_counts('location')

location
C     230
L     199
T     188
TL     78
CT     73
S      36
LS      4
C       1
Name: count, dtype: int64

In [45]:
test_namelist = get_loc_forpatients("./DiffusionSpinalMRISynthesis/Data_MRI/test_spinal_MRI")
test_namelist


,location,id
0,C,3544499
1,TL,3757848
2,L,1178180
3,C,2626171
4,TL,2454745
...,...,...
198,T,2106782
199,T,2813142
200,C,3734059
201,L,2751084


In [46]:
test_namelist.value_counts('location')

location
C     77
L     46
T     44
TL    17
CT    13
S      6
Name: count, dtype: int64

In [3]:
train_namelist = get_disease_forpatients("./DiffusionSpinalMRISynthesis/Data_MRI/train_spinal_MRI")
test_namelist = get_disease_forpatients("./DiffusionSpinalMRISynthesis/Data_MRI/test_spinal_MRI")
test_namelist

,disease,id
0,meningioma,3749258
1,ossifying_fibroma,3330108
2,M_lung,3239583
3,schwannoma,3755273
4,M_lung,2106782
...,...,...
643,ependymoma,2918664
742,teratoma,2751084
749,schwannoma,3646402
766,M_lung,2306367


In [9]:
test_namelist.query(f"disease == 'meninges_melanoma'")

,disease,id
186,meninges_melanoma,2863463


In [25]:
train_namelist['disease'] = train_namelist['disease'].apply(lambda x:'metastasis' if x.startswith("M_") else x)
test_namelist['disease'] = test_namelist['disease'].apply(lambda x:'metastasis' if x.startswith("M_") else x)
train_namelist

,disease,id
0,schwannoma,3392010
1,chordoma,841319
2,follicular_lymphoma,2293031
3,chondrosarcoma,2492374
4,schwannoma,3549764
...,...,...
3151,plasmacytoma,2479092
3450,schwannoma,744631
3478,tendonsheathGCT,2133004
3756,metastasis,3015655


In [26]:
train_disease_count = train_namelist.drop_duplicates('id').reset_index(drop=True).value_counts('disease')
test_disease_count = test_namelist.drop_duplicates('id').reset_index(drop=True).value_counts('disease')